In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np


Using TensorFlow backend.


In [2]:
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import load_model

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

sess = tf.Session()
graph = tf.get_default_graph()

# IMPORTANT: models have to be loaded AFTER SETTING THE SESSION for keras! 
# Otherwise, their weights will be unavailable in the threads after the session there has been set
set_session(sess)

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
class COGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/02/18/coupled_gan/"""
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.d1, self.d2 = self.build_discriminators()
        self.d1.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d2.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.g1, self.g2 = self.build_generators()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img1 = self.g1(z)
        img2 = self.g2(z)

        # For the combined model we will only train the generators
        self.d1.trainable = False
        self.d2.trainable = False

        # The valid takes generated images as input and determines validity
        valid1 = self.d1(img1)
        valid2 = self.d2(img2)

        # The combined model  (stacked generators and discriminators)
        # Trains generators to fool discriminators
        self.combined = Model(z, [valid1, valid2])
        self.combined.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
                                    optimizer=optimizer)

    def build_generators(self):

        # Shared weights between generators
        model = Sequential()
        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        noise = Input(shape=(self.latent_dim,))
        feature_repr = model(noise)

        # Generator 1
        g1 = Dense(1024)(feature_repr)
        g1 = LeakyReLU(alpha=0.2)(g1)
        g1 = BatchNormalization(momentum=0.8)(g1)
        g1 = Dense(np.prod(self.img_shape), activation='tanh')(g1)
        img1 = Reshape(self.img_shape)(g1)

        # Generator 2
        g2 = Dense(1024)(feature_repr)
        g2 = LeakyReLU(alpha=0.2)(g2)
        g2 = BatchNormalization(momentum=0.8)(g2)
        g2 = Dense(np.prod(self.img_shape), activation='tanh')(g2)
        img2 = Reshape(self.img_shape)(g2)

        model.summary()

        return Model(noise, img1), Model(noise, img2)

    def build_discriminators(self):

        img1 = Input(shape=self.img_shape)
        img2 = Input(shape=self.img_shape)

        # Shared discriminator layers
        model = Sequential()
        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))

        img1_embedding = model(img1)
        img2_embedding = model(img2)

        # Discriminator 1
        validity1 = Dense(1, activation='sigmoid')(img1_embedding)
        # Discriminator 2
        validity2 = Dense(1, activation='sigmoid')(img2_embedding)

        return Model(img1, validity1), Model(img2, validity2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Images in domain A and B (rotated)
        X1 = X_train[:int(X_train.shape[0]/2)]
        X2 = X_train[int(X_train.shape[0]/2):]
        X2 = scipy.ndimage.interpolation.rotate(X2, 90, axes=(1, 2))

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ----------------------
            #  Train Discriminators
            # ----------------------

            # Select a random batch of images
            idx = np.random.randint(0, X1.shape[0], batch_size)
            imgs1 = X1[idx]
            imgs2 = X2[idx]

            # Sample noise as generator input
            noise = np.random.normal(0, 1, (batch_size, 100))

            # Generate a batch of new images
            gen_imgs1 = self.g1.predict(noise)
            gen_imgs2 = self.g2.predict(noise)

            # Train the discriminators
            d1_loss_real = self.d1.train_on_batch(imgs1, valid)
            d2_loss_real = self.d2.train_on_batch(imgs2, valid)
            d1_loss_fake = self.d1.train_on_batch(gen_imgs1, fake)
            d2_loss_fake = self.d2.train_on_batch(gen_imgs2, fake)
            d1_loss = 0.5 * np.add(d1_loss_real, d1_loss_fake)
            d2_loss = 0.5 * np.add(d2_loss_real, d2_loss_fake)


            # ------------------
            #  Train Generators
            # ------------------

            g_loss = self.combined.train_on_batch(noise, [valid, valid])

            # Plot the progress
            print ("%d [D1 loss: %f, acc.: %.2f%%] [D2 loss: %f, acc.: %.2f%%] [G loss: %f]" \
                % (epoch, d1_loss[0], 100*d1_loss[1], d2_loss[0], 100*d2_loss[1], g_loss[0]))

            # If at save interval => save generated image samples
            if epoch>=29000:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 4, 4
        noise = np.random.normal(0, 1, (r * int(c/2), 100))
        gen_imgs1 = self.g1.predict(noise)
        gen_imgs2 = self.g2.predict(noise)

        gen_imgs = np.concatenate([gen_imgs1, gen_imgs2])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(r"C:\Users\rishi\Documents\COGAN Images\mnist_%d.png.png" % epoch)
        plt.close()


In [6]:
global sess
global graph
with graph.as_default():
    set_session(sess)
    cogan = COGAN()
    cogan.train(epochs=30000, batch_size=32, sample_interval=200)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)               1024      
_________________________________________________________________
dense_16 (Dense)             (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 512)               2048      
Total params: 160,512
Trainable params: 158,976
Non-trainable params: 1,536
____________________________________________

c:\users\rishi\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'



3 [D1 loss: 0.340155, acc.: 78.12%] [D2 loss: 0.350829, acc.: 75.00%] [G loss: 1.542177]
4 [D1 loss: 0.324665, acc.: 84.38%] [D2 loss: 0.332584, acc.: 82.81%] [G loss: 1.727338]
5 [D1 loss: 0.292810, acc.: 96.88%] [D2 loss: 0.317227, acc.: 87.50%] [G loss: 1.979931]
6 [D1 loss: 0.254389, acc.: 95.31%] [D2 loss: 0.272518, acc.: 92.19%] [G loss: 2.362729]
7 [D1 loss: 0.195922, acc.: 98.44%] [D2 loss: 0.222450, acc.: 100.00%] [G loss: 2.627196]
8 [D1 loss: 0.165214, acc.: 100.00%] [D2 loss: 0.200192, acc.: 100.00%] [G loss: 3.185672]
9 [D1 loss: 0.139158, acc.: 100.00%] [D2 loss: 0.148794, acc.: 100.00%] [G loss: 3.562829]
10 [D1 loss: 0.113990, acc.: 100.00%] [D2 loss: 0.116957, acc.: 100.00%] [G loss: 3.839445]
11 [D1 loss: 0.103594, acc.: 100.00%] [D2 loss: 0.099101, acc.: 100.00%] [G loss: 4.319899]
12 [D1 loss: 0.087671, acc.: 100.00%] [D2 loss: 0.097824, acc.: 100.00%] [G loss: 4.618989]
13 [D1 loss: 0.071864, acc.: 100.00%] [D2 loss: 0.081808, acc.: 100.00%] [G loss: 4.965909]
14 

94 [D1 loss: 0.136203, acc.: 93.75%] [D2 loss: 0.048540, acc.: 98.44%] [G loss: 10.604643]
95 [D1 loss: 0.056255, acc.: 96.88%] [D2 loss: 0.016675, acc.: 100.00%] [G loss: 12.021576]
96 [D1 loss: 0.066750, acc.: 96.88%] [D2 loss: 0.045707, acc.: 98.44%] [G loss: 12.746479]
97 [D1 loss: 0.099370, acc.: 93.75%] [D2 loss: 0.029997, acc.: 100.00%] [G loss: 11.494175]
98 [D1 loss: 0.065605, acc.: 95.31%] [D2 loss: 0.031068, acc.: 98.44%] [G loss: 11.130921]
99 [D1 loss: 0.080903, acc.: 95.31%] [D2 loss: 0.044192, acc.: 100.00%] [G loss: 9.930332]
100 [D1 loss: 0.027682, acc.: 100.00%] [D2 loss: 0.051357, acc.: 98.44%] [G loss: 11.044733]
101 [D1 loss: 0.051823, acc.: 100.00%] [D2 loss: 0.012750, acc.: 100.00%] [G loss: 10.878528]
102 [D1 loss: 0.189252, acc.: 90.62%] [D2 loss: 0.088157, acc.: 96.88%] [G loss: 12.794557]
103 [D1 loss: 0.766894, acc.: 64.06%] [D2 loss: 0.457246, acc.: 81.25%] [G loss: 8.728722]
104 [D1 loss: 0.410662, acc.: 82.81%] [D2 loss: 0.077444, acc.: 96.88%] [G loss: 1

276 [D1 loss: 0.626755, acc.: 54.69%] [D2 loss: 0.653949, acc.: 54.69%] [G loss: 1.618892]
277 [D1 loss: 0.645354, acc.: 54.69%] [D2 loss: 0.711483, acc.: 43.75%] [G loss: 1.588362]
278 [D1 loss: 0.633370, acc.: 50.00%] [D2 loss: 0.674841, acc.: 45.31%] [G loss: 1.687625]
279 [D1 loss: 0.652584, acc.: 51.56%] [D2 loss: 0.715574, acc.: 43.75%] [G loss: 1.674517]
280 [D1 loss: 0.675145, acc.: 46.88%] [D2 loss: 0.632769, acc.: 59.38%] [G loss: 1.710748]
281 [D1 loss: 0.675573, acc.: 39.06%] [D2 loss: 0.736953, acc.: 42.19%] [G loss: 1.525375]
282 [D1 loss: 0.685970, acc.: 40.62%] [D2 loss: 0.659241, acc.: 53.12%] [G loss: 1.571163]
283 [D1 loss: 0.654265, acc.: 40.62%] [D2 loss: 0.679404, acc.: 46.88%] [G loss: 1.570425]
284 [D1 loss: 0.668123, acc.: 51.56%] [D2 loss: 0.637574, acc.: 54.69%] [G loss: 1.680425]
285 [D1 loss: 0.657066, acc.: 56.25%] [D2 loss: 0.693397, acc.: 50.00%] [G loss: 1.588199]
286 [D1 loss: 0.649709, acc.: 53.12%] [D2 loss: 0.620763, acc.: 65.62%] [G loss: 1.739512]

...

29615 [D1 loss: 0.585499, acc.: 67.19%] [D2 loss: 0.548610, acc.: 78.12%] [G loss: 2.389299]
29616 [D1 loss: 0.590993, acc.: 65.62%] [D2 loss: 0.649429, acc.: 64.06%] [G loss: 2.546054]
29617 [D1 loss: 0.670924, acc.: 54.69%] [D2 loss: 0.615894, acc.: 59.38%] [G loss: 2.479678]
29618 [D1 loss: 0.661217, acc.: 62.50%] [D2 loss: 0.524915, acc.: 67.19%] [G loss: 2.585067]
29619 [D1 loss: 0.658865, acc.: 62.50%] [D2 loss: 0.568869, acc.: 70.31%] [G loss: 2.339268]
29620 [D1 loss: 0.703699, acc.: 54.69%] [D2 loss: 0.585525, acc.: 67.19%] [G loss: 2.497547]
29621 [D1 loss: 0.564688, acc.: 75.00%] [D2 loss: 0.636290, acc.: 73.44%] [G loss: 2.497187]
29622 [D1 loss: 0.646562, acc.: 62.50%] [D2 loss: 0.608648, acc.: 65.62%] [G loss: 2.449199]
29623 [D1 loss: 0.719467, acc.: 46.88%] [D2 loss: 0.536278, acc.: 67.19%] [G loss: 2.427122]
29624 [D1 loss: 0.658481, acc.: 54.69%] [D2 loss: 0.614762, acc.: 68.75%] [G loss: 2.303606]
29625 [D1 loss: 0.590247, acc.: 65.62%] [D2 loss: 0.637297, acc.: 56.2

29704 [D1 loss: 0.609723, acc.: 60.94%] [D2 loss: 0.694955, acc.: 57.81%] [G loss: 2.441979]
29705 [D1 loss: 0.615544, acc.: 65.62%] [D2 loss: 0.646871, acc.: 62.50%] [G loss: 2.526299]
29706 [D1 loss: 0.637699, acc.: 60.94%] [D2 loss: 0.461490, acc.: 84.38%] [G loss: 2.422058]
29707 [D1 loss: 0.667506, acc.: 57.81%] [D2 loss: 0.622594, acc.: 65.62%] [G loss: 2.505217]
29708 [D1 loss: 0.630748, acc.: 65.62%] [D2 loss: 0.672238, acc.: 64.06%] [G loss: 2.376498]
29709 [D1 loss: 0.605597, acc.: 68.75%] [D2 loss: 0.631606, acc.: 64.06%] [G loss: 2.491603]
29710 [D1 loss: 0.602636, acc.: 70.31%] [D2 loss: 0.631728, acc.: 59.38%] [G loss: 2.595302]
29711 [D1 loss: 0.549451, acc.: 70.31%] [D2 loss: 0.544240, acc.: 76.56%] [G loss: 2.752868]
29712 [D1 loss: 0.615787, acc.: 60.94%] [D2 loss: 0.720575, acc.: 54.69%] [G loss: 2.319395]
29713 [D1 loss: 0.693922, acc.: 56.25%] [D2 loss: 0.647811, acc.: 57.81%] [G loss: 2.474995]
29714 [D1 loss: 0.613088, acc.: 68.75%] [D2 loss: 0.615562, acc.: 65.6

29793 [D1 loss: 0.611309, acc.: 65.62%] [D2 loss: 0.526439, acc.: 70.31%] [G loss: 2.696308]
29794 [D1 loss: 0.667262, acc.: 57.81%] [D2 loss: 0.574759, acc.: 67.19%] [G loss: 2.385551]
29795 [D1 loss: 0.607363, acc.: 65.62%] [D2 loss: 0.688324, acc.: 60.94%] [G loss: 2.473909]
29796 [D1 loss: 0.639434, acc.: 68.75%] [D2 loss: 0.557610, acc.: 76.56%] [G loss: 2.393311]
29797 [D1 loss: 0.594307, acc.: 64.06%] [D2 loss: 0.593998, acc.: 64.06%] [G loss: 2.385200]
29798 [D1 loss: 0.620941, acc.: 64.06%] [D2 loss: 0.538754, acc.: 75.00%] [G loss: 2.871013]
29799 [D1 loss: 0.618634, acc.: 67.19%] [D2 loss: 0.522584, acc.: 68.75%] [G loss: 2.604263]
29800 [D1 loss: 0.704600, acc.: 53.12%] [D2 loss: 0.641310, acc.: 64.06%] [G loss: 2.426776]
29801 [D1 loss: 0.678342, acc.: 51.56%] [D2 loss: 0.675167, acc.: 57.81%] [G loss: 2.431168]
29802 [D1 loss: 0.728702, acc.: 54.69%] [D2 loss: 0.678122, acc.: 56.25%] [G loss: 2.236984]
29803 [D1 loss: 0.646151, acc.: 57.81%] [D2 loss: 0.590653, acc.: 67.1

29882 [D1 loss: 0.730078, acc.: 54.69%] [D2 loss: 0.612614, acc.: 64.06%] [G loss: 2.488120]
29883 [D1 loss: 0.623062, acc.: 60.94%] [D2 loss: 0.572139, acc.: 68.75%] [G loss: 2.435250]
29884 [D1 loss: 0.580292, acc.: 67.19%] [D2 loss: 0.579900, acc.: 73.44%] [G loss: 2.445065]
29885 [D1 loss: 0.559861, acc.: 71.88%] [D2 loss: 0.561175, acc.: 81.25%] [G loss: 2.675421]
29886 [D1 loss: 0.568637, acc.: 70.31%] [D2 loss: 0.711837, acc.: 56.25%] [G loss: 2.595317]
29887 [D1 loss: 0.690071, acc.: 51.56%] [D2 loss: 0.485015, acc.: 81.25%] [G loss: 2.557251]
29888 [D1 loss: 0.499707, acc.: 75.00%] [D2 loss: 0.617841, acc.: 67.19%] [G loss: 2.479093]
29889 [D1 loss: 0.638054, acc.: 60.94%] [D2 loss: 0.622020, acc.: 67.19%] [G loss: 2.388793]
29890 [D1 loss: 0.662614, acc.: 68.75%] [D2 loss: 0.552879, acc.: 68.75%] [G loss: 2.427173]
29891 [D1 loss: 0.572903, acc.: 71.88%] [D2 loss: 0.572780, acc.: 71.88%] [G loss: 2.605417]
29892 [D1 loss: 0.614506, acc.: 67.19%] [D2 loss: 0.667886, acc.: 56.2

29971 [D1 loss: 0.611526, acc.: 68.75%] [D2 loss: 0.598578, acc.: 65.62%] [G loss: 2.447092]
29972 [D1 loss: 0.542811, acc.: 67.19%] [D2 loss: 0.612685, acc.: 59.38%] [G loss: 2.530955]
29973 [D1 loss: 0.622862, acc.: 70.31%] [D2 loss: 0.616334, acc.: 62.50%] [G loss: 2.443889]
29974 [D1 loss: 0.628520, acc.: 64.06%] [D2 loss: 0.506609, acc.: 78.12%] [G loss: 2.334645]
29975 [D1 loss: 0.650295, acc.: 59.38%] [D2 loss: 0.541608, acc.: 71.88%] [G loss: 2.280641]
29976 [D1 loss: 0.688725, acc.: 56.25%] [D2 loss: 0.655464, acc.: 64.06%] [G loss: 2.316570]
29977 [D1 loss: 0.643389, acc.: 67.19%] [D2 loss: 0.505232, acc.: 76.56%] [G loss: 2.619788]
29978 [D1 loss: 0.532549, acc.: 75.00%] [D2 loss: 0.642206, acc.: 68.75%] [G loss: 2.218351]
29979 [D1 loss: 0.727010, acc.: 56.25%] [D2 loss: 0.661691, acc.: 67.19%] [G loss: 2.379775]
29980 [D1 loss: 0.583651, acc.: 70.31%] [D2 loss: 0.674536, acc.: 60.94%] [G loss: 2.348611]
29981 [D1 loss: 0.526169, acc.: 73.44%] [D2 loss: 0.593925, acc.: 70.3

In [ ]:
def generate_images(cnt):
    c=1
    while(c <= cnt):
        noise = np.random.normal(0, 1, (1, 100))
        #noise = np.random.normal(0, 1, (1, 100))
        #gen_imgs = .generator.predict(noise)
        gen_imgs1 = cogan.g1.predict(noise)
        gen_imgs2 = cogan.g2.predict(noise)
        gen_imgs = np.concatenate([gen_imgs1, gen_imgs2])
        gen_imgs = 0.5 * gen_imgs + 0.5
        plt.imshow(gen_imgs[0, :,:,0], cmap='gray')
        plt.axis('off')
        plt.savefig(r"C:\Users\rishi\Documents\COGAN Images\generated_cogan\cogan_{}.jpg".format(c), bbox_inches='tight', pad_inches=-0.0005)
        print("Image {} saved.".format(c))
        c+=1
        plt.close()
generate_images(10000)